In [7]:
import os
import pandas as pd
import numpy as np

class market():
    def word_cut(self,documents):
        stopwords = self.stopwords
        import jieba
        texts = []
        for line in documents:
            words = ' '.join(jieba.cut(line)).split(' ') # 用空格去连接，连接后马上又拆分
            text = []
            for word in words:
                if (word not in stopwords) & (word != '')& (word != '\u3000')& (word != '\n')&(word != '\u200b'):
                    text.append(word)
            texts.append(text)
        self.docLength = len(documents)
        return(texts)
    def get_docLength(self):
        return(self.docLength)
    def frequency(self,texts,freq):
        from collections import defaultdict
        frequency = defaultdict(int) # value为int
        for text in texts:
            for word in text:
                frequency[word] += 1
        texts = [[word for word in text if frequency[word] > freq] for text in texts]
        return(texts)
    def regroup(self,texts):
        new_texts = []
        for i,sentence in enumerate(texts):
            new_texts.append(" ".join(sentence))
        return(new_texts)
    def add_stopwords(self,path):
        stopwords = set()
        with open(path,'r',encoding = 'cp936') as file:
            for line in file:
                stopwords.add(line.strip())
        self.stopwords = stopwords
        print("Load %s stopwords" %len(stopwords))
    def dictionary(self,docs):
        token_index ={}
        for sample in docs:
            for word in sample:
                if word not in token_index:
                    token_index[word] = len(token_index) + 1
        return(token_index)
    def count(self,docs):
        token_length ={}
        for sample in docs:
            for word in sample:
                if word not in token_length:
                    token_length[word] = 1
                else:
                    token_length[word] += 1
        return(token_length)
    def recoding(self,docs,token_index):
        for i,sample in enumerate(docs):
            for j,word in enumerate(sample):
                if word not in token_index:
                    sample[j] = -1
                else:
                    sample[j] = token_index[word]
            docs[i] = sample
        return(docs)
    def delete(self,docs):
        for index in range(len(docs)):
            for i in range(len(docs[index])-1,-1,-1):
                if docs[index][i] == -1:
                    docs[index].pop(i)
        return docs
    def random_pick(self,df,n):
        import random
        import numpy as np
        rand = np.arange(0,(len(df)-1),1)
        random.shuffle(rand)
        rand = list(rand[:n])
        df = df.loc[rand,]
        return(df)
    def read_vectors(self,path, topn):  # read top n word vectors, i.e. top is 10000
        lines_num, dim = 0, 0
        vectors = {}
        iw = []
        wi = {}
        with open(path, encoding='utf-8', errors='ignore') as f:
            first_line = True
            for line in f:
                if first_line:
                    first_line = False
                    dim = int(line.rstrip().split()[1]) # 删除向量末尾的空格，然后以空格拆分获得向量
                    continue
                lines_num += 1
                tokens = line.rstrip().split(' ')
                vectors[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])# 当数据源是ndarray时，asarray不会占用新的内存；当数据源不是ndarray,asarray与array一样
                iw.append(tokens[0]) # iw储存了所有的tokons[0]，意思是index_word
                if topn != 0 and lines_num >= topn:
                    break
        for i, w in enumerate(iw):
            wi[w] = i # wi是iw的反转，意思是word_index,用w来储存字符，用一个integer去给字符编码
        self.dim = dim
        self.max_words = topn
        self.word_index = wi
        self.index_word = iw
        self.vectors = vectors
        print("Load %s word vectors." % len(vectors))
    def embedding_matrix(self):
        embedding_matrix = np.zeros((self.max_words,self.dim))
        for word,i in self.word_index.items():
            if i < self.max_words:
                embedding_vector = self.vectors.get(word)
                if embedding_vector is not None:
                    embedding_matrix[i] = embedding_vector
        return embedding_matrix
    def navie_knn(self,dataSet, query, k):  
        # 计算出某一样本与所有样本的距离，选择最大(应该修改为最小？)的k个样本作为用于knn
        numSamples = dataSet.shape[0] # return row(sample) number of dataset

        ## step 1: calculate Euclidean distance  
        diff = np.tile(query, (numSamples, 1)) - dataSet #tile: 把query这个向量纵向复制，使得结果与dataset具有同样的行数
        squaredDiff = diff ** 2  
        squaredDist = np.sum(squaredDiff, axis = 1) # sum is performed by row  

        ## step 2: sort the distance  
        sortedDistIndices = np.argsort(squaredDist)   # numpy.argsort 返回的是数组值从小到大的索引值（注意是索引值，不是绝对值）
        if k > len(sortedDistIndices):  
            k = len(sortedDistIndices)  

        return sortedDistIndices[0:k]
    # build a big graph (normalized weight matrix)  
    def buildGraph(self,MatX, kernel_type, rbf_sigma = None, knn_num_neighbors = None):  
        num_samples = MatX.shape[0]  # return row(sample) number of MatX
        affinity_matrix = np.zeros((num_samples, num_samples), np.float32)  
        if kernel_type == 'rbf':  
            if rbf_sigma == None:  
                raise ValueError('You should input a sigma of rbf kernel!')  
            for i in range(num_samples):  
                row_sum = 0.0  
                for j in range(num_samples):  
                    diff = MatX[i, :] - MatX[j, :]  
                    affinity_matrix[i][j] = np.exp(sum(diff**2) / (-2.0 * rbf_sigma**2))  
                    row_sum += affinity_matrix[i][j]  
                affinity_matrix[i][:] /= row_sum  
        elif kernel_type == 'knn':  
            if knn_num_neighbors == None:  
                raise ValueError('You should input a k of knn kernel!')  
            for i in range(num_samples):  
                k_neighbors = self.navie_knn(MatX, MatX[i, :], knn_num_neighbors)  
                affinity_matrix[i][k_neighbors] = 1.0 / knn_num_neighbors  # 将节点i与附近的k个节点连接起来，每个边的权重是1/knn_num_neighbors
        else:  
            raise NameError('Not support kernel type! You can use knn or rbf!')  

        return affinity_matrix  
    # label propagation  
    def labelPropagation(self,Mat_Label, Mat_Unlabel, labels, kernel_type = 'rbf', rbf_sigma = 0.20, \
                        knn_num_neighbors = 10, max_iter = 500, tol = 1e-3):  
        # initialize  
        num_label_samples = Mat_Label.shape[0]  #已经标记的sample number
        num_unlabel_samples = Mat_Unlabel.shape[0]  #未标记的sample number
        num_samples = num_label_samples + num_unlabel_samples
        labels_list = np.unique(labels)  #有哪些label
        num_classes = len(labels_list)  #label的种类数

        MatX = np.vstack((Mat_Label, Mat_Unlabel))
        clamp_data_label = np.zeros((num_label_samples, num_classes), np.float32)  
        for i in range(num_label_samples):  
            clamp_data_label[i][labels[i]] = 1.0   #标记出每一个labelled sample的具体label是什么

        label_function = np.zeros((num_samples, num_classes), np.float32)  
        label_function[0 : num_label_samples] = clamp_data_label  
        label_function[num_label_samples : num_samples] = -1  

        # graph construction  
        affinity_matrix = self.buildGraph(MatX, kernel_type, rbf_sigma, knn_num_neighbors)  

        # start to propagation  
        iter = 0; pre_label_function = np.zeros((num_samples, num_classes), np.float32)  
        changed = np.abs(pre_label_function - label_function).sum()  
        while iter < max_iter and changed > tol:  
            if iter % 1 == 0:  
                print ("---> Iteration %d/%d, changed: %f" % (iter, max_iter, changed))
            pre_label_function = label_function  
            iter += 1  

            # propagation  
            label_function = np.dot(affinity_matrix, label_function)  

            # clamp  
            label_function[0 : num_label_samples] = clamp_data_label  

            # check converge  
            changed = np.abs(pre_label_function - label_function).sum()  

        # get terminate label of unlabeled data  
        unlabel_data_labels = np.zeros(num_unlabel_samples)  
        for i in range(num_unlabel_samples):  
            unlabel_data_labels[i] = np.argmax(label_function[i+num_label_samples]) #取出参数中元素最大值所对应的索引 

        return unlabel_data_labels 

In [9]:
process = market()
process.add_stopwords("D:/Users/PYTHON/Precision-Marketing/stopwords.txt")
process.read_vectors("D:/NLP/sgns.target.word-word.dynwin5.thr10.neg5.dim300.txt",10000)

Load 2316 stopwords
Load 10000 word vectors.


In [12]:
embedding_matrix = process.embedding_matrix()
embedding_matrix.shape

(10000, 300)

In [13]:
os.chdir("D:/Users/PYTHON/Precision-Marketing")

WORRY = pd.read_csv("已标记的担忧咋办标签.csv")
WORRY = WORRY[["博文","是否担忧（1=担忧，0=不担忧）"]] 
PATIENT = pd.read_csv("已标记的会不会得病标签.csv")
PATIENT = PATIENT[["博文","是否担忧（1=担忧，0=不担忧）"]]

print(WORRY.head(5))
print(PATIENT.head(5))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                                                  博文  是否担忧（1=担忧，0=不担忧）
0  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
1  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...                 0
2  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
3  问：如何让猫黏上你？答：穿一条毛茸茸的裤子。————但是我现在有点担心你现在睡这么多晚上我睡...                 0
4  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
                                                  博文  是否担忧（1=担忧，0=不担忧）
0  给好多人说了新年快乐说了新年祝福还没有给自己说过。新年快乐希望在2020年你可以学有所成重要...               0.0
1  2019开始倒数了哦原来小时候觉得很遥远的2020年就是明天了要说2019年我收获了什么呢？...               1.0
2  微信，QQ都是认识的人，没有办法发泄内心的不满，渐渐养成了不发朋友圈的习惯，但是自己变得越来...               1.0
3  现在已经对酒上瘾很严重每天这个点必须喝半杯而且一天喝水喝特别少大概就一瓶不到..我会不会得病...               1.0
4  这段时间每天睡得晚，起的晚，可恨没有好好吃早餐。会不会得病啊中午上一节男老师的尊巴舞，如果体...               1.0


In [14]:
df = pd.concat([WORRY,PATIENT]) # concat之后row index会重复
# df = pd.concat([PATIENT])
df.reset_index(drop = True,inplace = True)
print(len(df))
df = df.dropna()
print(len(df))
df.head(5)

1049497
1636


,博文,是否担忧（1=担忧，0=不担忧）
0,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0
1,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...,0.0
2,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0
3,问：如何让猫黏上你？答：穿一条毛茸茸的裤子。————但是我现在有点担心你现在睡这么多晚上我睡...,0.0
4,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0


In [15]:
df["是否担忧（1=担忧，0=不担忧）"].value_counts()

0.0    1370
1.0     266
Name: 是否担忧（1=担忧，0=不担忧）, dtype: int64

In [62]:
df_worry = df[df["是否担忧（1=担忧，0=不担忧）"] == 1]
df_worry.reset_index(drop = True,inplace = True)
df_non_worry = df[df["是否担忧（1=担忧，0=不担忧）"] == 0]
df_non_worry.reset_index(drop = True,inplace = True)
df_non_worry = process.random_pick(df_non_worry,min(len(df_worry),len(df_non_worry)))
print(len(df_worry))
print(len(df_non_worry))

266
266


In [63]:
df_worry

,博文,是否担忧（1=担忧，0=不担忧）
0,焦虑症我咋办啊!!我又开始怀疑自己心脏病了…每时每刻都在担心自己是不是会突然倒下去这几天都...,1.0
1,累…要做好工作还要当好妈妈，有多久没有凌晨两点前睡过觉了，刚收拾完宝贝明天去海洋馆要拿的东西...,1.0
2,我是真的很担心我的胳膊不好使了咋办2武汉·亿童大厦​,1.0
3,臭宝前天晚上去医院看了，大夫说是支原体感染，在网上查说支原体感染很麻烦，很容易复发，我又担心...,1.0
4,36个年头，第一次这么怕死，虽然问过医生，还是有点担心，怕真有什么，俩宝咋办，四老咋办，唉，...,1.0
...,...,...
261,刚刚收到公司行政部的短信调查，问我感冒咳嗽好了没。啊啊啊，好害怕回去上班！路上好危险，要搭地...,1.0
262,你们在家里呆有没有出现很闷 就是心慌然后呼吸急促 怎么办我好害怕😨 ​,1.0
263,妈妈不在家的时候我总是很焦虑，我害怕她出事，虽然她身体健康，我想和她每天每个时间都在一起，妈...,1.0
264,望着躺在重症监护室里的父亲，心里五味杂陈，我笑着告诉他一切都好，却害怕他看见我眼角的泪有些话...,1.0


In [64]:
df_worry

,博文,是否担忧（1=担忧，0=不担忧）
0,焦虑症我咋办啊!!我又开始怀疑自己心脏病了…每时每刻都在担心自己是不是会突然倒下去这几天都...,1.0
1,累…要做好工作还要当好妈妈，有多久没有凌晨两点前睡过觉了，刚收拾完宝贝明天去海洋馆要拿的东西...,1.0
2,我是真的很担心我的胳膊不好使了咋办2武汉·亿童大厦​,1.0
3,臭宝前天晚上去医院看了，大夫说是支原体感染，在网上查说支原体感染很麻烦，很容易复发，我又担心...,1.0
4,36个年头，第一次这么怕死，虽然问过医生，还是有点担心，怕真有什么，俩宝咋办，四老咋办，唉，...,1.0
...,...,...
261,刚刚收到公司行政部的短信调查，问我感冒咳嗽好了没。啊啊啊，好害怕回去上班！路上好危险，要搭地...,1.0
262,你们在家里呆有没有出现很闷 就是心慌然后呼吸急促 怎么办我好害怕😨 ​,1.0
263,妈妈不在家的时候我总是很焦虑，我害怕她出事，虽然她身体健康，我想和她每天每个时间都在一起，妈...,1.0
264,望着躺在重症监护室里的父亲，心里五味杂陈，我笑着告诉他一切都好，却害怕他看见我眼角的泪有些话...,1.0


In [65]:
df_non_worry

,博文,是否担忧（1=担忧，0=不担忧）
1253,连续快一周，睡不好，总是这个点醒来，发现胸口闷的难受，这样下去不知道会不会得病三年了，每次的...,0.0
201,越到考试时间就越佛我现在只担心我考试的时候想睡觉咋办我想着去买风油精的​,0.0
1242,王源#王源2019无畏前行#王源，越来越想抱抱你（妈妈的抱抱），再想下去我会不会得病啊！@...,0.0
683,现在不是担心大鼠手感害不害怕和我手小抓得住抓不住的问题，现在比较担心，造模过程后期我的大鼠都...,0.0
140,#2020年不用房地产短期刺激经济#不管咋样至少2019对我还不错最担心的按揭贷款办下来了​,0.0
...,...,...
1172,大家应该都不会取关之前的类似的这种公众号吧唉昨天报完道本来以为换到了一个三人间还是加上一个师...,0.0
150,我先生问过我“如果卖不出去，咋办？”实话说，我真的从来没担心过～看，朋友的朋友都找我买了，我...,0.0
37,我常常担心小F以后万一生女儿了眼睛像他可咋办啊！（我也不知道我心思这个干嘛也不是我女儿）希望...,0.0
631,又到外地了，这次的考核完全没底，咋个办办好，担心自己的同时还要担心家里那口，天气这么冷，大晚...,0.0


In [66]:
df_worry = df_worry.dropna()
df_non_worry = df_non_worry.dropna()
print(len(df_worry))
print(len(df_non_worry))

df_use = pd.concat([df_worry,df_non_worry])
df_use.reset_index(drop = True,inplace = True)
df_use = df_use.reindex(np.random.permutation(df_use.index))
df_use.head()


266
266


,博文,是否担忧（1=担忧，0=不担忧）
348,#老人出行是否应避开高峰期#每次早高峰挤公交遇到有不花钱就可以坐车的敬老卡就特别担心老人家被...,0.0
124,前两周军训都住学校里，没发现自己用纸那么猛，回家才发觉，我不受控制一直抽纸巾，一点污渍或是水...,1.0
237,#总怀疑自己得病怎么办#昨晚梦见自己一个人上街没带口罩，回家之后哭着和妈妈说会不会得新型肺炎 ​,1.0
469,看有人说国内不先交10万，不给抢救，致死。这种事情我没看到过也不知道是不是真的，但我想如果你...,0.0
416,俺圆满啦还看到一颗流星被向导安利了个APP直接用手机拍麻麻再也不用担心我单反充电器坏了咋办了​,0.0


In [67]:
x_train = process.word_cut(df_use["博文"])
x_train = process.frequency(x_train,5)

In [68]:
token_index = process.dictionary(x_train)

In [69]:
token_length = process.count(x_train)
token_length = {key:token_length[key] for key in sorted(token_length,key = lambda x: token_length[x],reverse = True)[:round((2/5)*len(token_index))]}

In [70]:
# 给字符编码。如果x_train中的词语不在传入的实参token_index中，那么就编码为-1
x_train = process.recoding(x_train,process.word_index)
x_train[:5]

[[2143,
  -1,
  2405,
  1117,
  -1,
  560,
  3624,
  -1,
  6568,
  -1,
  -1,
  -1,
  2143,
  -1,
  -1,
  -1,
  -1,
  2799,
  -1,
  6917,
  -1,
  374,
  -1,
  -1,
  2799,
  -1,
  -1,
  6917,
  693,
  -1,
  785,
  116,
  54,
  377,
  80,
  8003,
  374,
  -1,
  -1,
  2799,
  -1,
  -1,
  6917,
  693,
  1396,
  439,
  7198,
  118,
  2117,
  -1,
  1990,
  -1,
  7798,
  1947],
 [136,
  1645,
  141,
  117,
  1034,
  2436,
  1381,
  1534,
  472,
  -1,
  117,
  3456,
  2493,
  -1,
  6935,
  3132,
  -1,
  1381,
  -1,
  6935,
  117,
  225,
  491,
  462,
  -1,
  -1,
  7798,
  1947],
 [449, 3829, -1, -1, -1, 2436, 4502, 2696, -1],
 [1122,
  118,
  116,
  855,
  1794,
  1034,
  2966,
  462,
  2456,
  589,
  179,
  2456,
  2322,
  673,
  527,
  2678],
 [1959, 2974, 6209, 2777, 3624, 5147, -1]]

In [71]:
x_train = process.delete(x_train)
x_train[:2]

[[2143,
  2405,
  1117,
  560,
  3624,
  6568,
  2143,
  2799,
  6917,
  374,
  2799,
  6917,
  693,
  785,
  116,
  54,
  377,
  80,
  8003,
  374,
  2799,
  6917,
  693,
  1396,
  439,
  7198,
  118,
  2117,
  1990,
  7798,
  1947],
 [136,
  1645,
  141,
  117,
  1034,
  2436,
  1381,
  1534,
  472,
  117,
  3456,
  2493,
  6935,
  3132,
  1381,
  6935,
  117,
  225,
  491,
  462,
  7798,
  1947]]

In [72]:
import keras
import tensorflow
from keras import preprocessing

max_len = 50
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen = max_len)

x_train.shape

(532, 50)

In [73]:
y_train = df_use[["是否担忧（1=担忧，0=不担忧）"]]
# y_train["是否担忧（1=担忧，0=不担忧）"] = y_train["是否担忧（1=担忧，0=不担忧）"].astype("category")
y_train.head(5)

,是否担忧（1=担忧，0=不担忧）
348,0.0
124,1.0
237,1.0
469,0.0
416,0.0


In [74]:
y_in = len(y_train)
y_train = np.array(y_train)
y_train = y_train.reshape(y_in)
y_train[:5]

array([0., 1., 1., 0., 0.])

In [76]:
print(x_train.shape)
print(y_train.shape)

(532, 50)
(532,)


In [77]:
def get_values(token_index):
    values = []
    for key in token_index:
        values.append(token_index[key])
    return(values)
values = get_values(token_index)
values[:5]
print("最大的序号是：",max(values))

最大的序号是： 663


In [78]:
from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding,LSTM,Bidirectional,Dropout

max_features = 10000
max_len = 50

model = Sequential()
model.add(Embedding(max_features,300,input_length = max_len,mask_zero = True)) # 遇到0，就不会反向传播更新权重
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(20),merge_mode = 'concat'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 300)           3000000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 50, 300)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 40)                51360     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 41        
Total params: 3,051,401
Trainable params: 3,051,401
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [80]:
history = model.fit(x_train,
                    y_train,
                    epochs = 10,
                    batch_size = 128, # batch_size越大越好，但是太大会影响计算效率
                    validation_split= 0)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
532/532 [==============================] - 1s 2ms/step - loss: 0.6909 - acc: 0.5226
Epoch 2/10
532/532 [==============================] - 1s 1ms/step - loss: 0.6250 - acc: 0.7124
Epoch 3/10
532/532 [==============================] - 1s 1ms/step - loss: 0.5647 - acc: 0.8158
Epoch 4/10
532/532 [==============================] - 1s 1ms/step - loss: 0.5078 - acc: 0.8195
Epoch 5/10
532/532 [==============================] - 1s 1ms/step - loss: 0.4570 - acc: 0.8459
Epoch 6/10
532/532 [==============================] - 1s 1ms/step - loss: 0.3974 - acc: 0.8534
Epoch 7/10
532/532 [==============================] - 1s 1ms/step - loss: 0.3815 - acc: 0.8553
Epoch 8/10
532/532 [==============================] - 1s 1ms/step - loss: 0.3410 - acc: 0.8910
Epoch 9/10
532/532 [==============================] - 1s 1ms/step - loss: 0.3229 - acc: 0.8778
Epoch 10/10
532/532 [==============================] - 1s 1ms/step - loss: 0.3242 - acc: 0.8741


In [103]:
# model.save("Bi-LSTM(加入预训练的词向量库).h5")

In [94]:
new_data = pd.read_csv("未标记的担忧咋办标签.csv")
new_data_preserved = new_data
new_data.head(2)

,用户名,博文,预测是否担忧（1=担忧，0=不担忧）,转发数,评论数,点赞数,发文时间,来自,页面网址,博文链接
0,玥玥的碎花小裙儿,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0,,,,\n 2019年12月31日 23:52\n ...,realme Q 四摄迅猛龙,https://s.weibo.com/weibo/%25E6%258B%2585%25E5...,https://weibo.com/6275719793/Innh3oI0n?refer_f...
1,海里星星16687,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...,0,,,,\n 2019年12月31日 23:35\n ...,OPPO超视野全面屏R15,https://s.weibo.com/weibo/%25E6%258B%2585%25E5...,https://weibo.com/6607786597/InnadlLNd?refer_f...


In [95]:
new_data = new_data[["博文"]]
new_data["博文"] = process.word_cut(new_data["博文"])
new_data.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,博文
0,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."
1,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."


In [96]:
new_data["博文"] = process.frequency(new_data["博文"],5)
new_data.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,博文
0,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."
1,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."


In [97]:
new_data["博文"] = process.recoding(new_data["博文"],process.word_index)
new_data.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,博文
0,"[-1, -1, -1, -1, -1, -1, 4344, -1, -1, 3076, -..."
1,"[-1, -1, -1, -1, -1, -1, 4344, -1, -1, 3076, -..."


In [98]:
new_data["博文"] = process.delete(new_data["博文"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
x_test = preprocessing.sequence.pad_sequences(new_data["博文"],maxlen = max_len)

In [100]:
prediction = model.predict(x_test)
prediction = prediction.reshape(len(prediction))
prediction

array([0.03170225, 0.04756665, 0.03170225, ..., 0.02784744, 0.23346764,
       0.03607011], dtype=float32)

In [102]:
Result = pd.DataFrame({"预测结果":list(prediction),"博文":new_data_preserved["博文"]})
# Result.to_excel("Bi-LSTM预测结果（(加入预训练的词向量库)）.xlsx",header = True)